<a href="https://colab.research.google.com/github/itisWasp/vsit-rwanda-sentiment/blob/main/new_visit_rwanda_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# #VisitRwanda Sentiment Analysis

## 1.Authenticate to Twitter

In [67]:
#!pip install pandas-datareader
import pandas as pd
import numpy as np
import tweepy as tw
import matplotlib.pyplot as plt

In [68]:
#Twitter API connection
consumer_key = 'ZUJtVbQsnlhjwPppB96Iy1zXn'
consumer_secret = 'xK2BdKbAP0o6KqLGrXRzJdfmOpOf4XaKhGhFfEs7MBiBj9ctQK'
access_token = '1437342555151249408-NSpHFFQF3m2fDEXwMg0pyUXOOBr0J4'
access_token_secret ='0rheoIeGWe8ICV6fL3NKljDom7g8fCZyt4JnAjwBPvAzc'

In [69]:
#Authenticate credentials
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

## 2.Get Tweets

In [70]:
hashtag='#visitrwanda'
query = tw.Cursor(api.search, q=hashtag).items(10000)
tweets = [{'Tweets': tweet.text, 'TimeStamp': tweet.created_at} for tweet in query]
print(tweets)

[{'Tweets': 'RT @fatoulo11: I enjoy watching my guests fall in love with #Rwanda as I did years ago. Their excitement says it all. The best I have heard…', 'TimeStamp': datetime.datetime(2021, 11, 29, 10, 5, 56)}, {'Tweets': 'RT @fatoulo11: I enjoy watching my guests fall in love with #Rwanda as I did years ago. Their excitement says it all. The best I have heard…', 'TimeStamp': datetime.datetime(2021, 11, 29, 9, 51, 47)}, {'Tweets': 'RT @fatoulo11: I enjoy watching my guests fall in love with #Rwanda as I did years ago. Their excitement says it all. The best I have heard…', 'TimeStamp': datetime.datetime(2021, 11, 29, 9, 49, 44)}, {'Tweets': '@Mondo73410452 Good and thanks for your congrats, On you point of misunderstanding  allow me to request you to come… https://t.co/iU5HmVp3Ad', 'TimeStamp': datetime.datetime(2021, 11, 29, 9, 35, 39)}, {'Tweets': '#VisitRwanda https://t.co/96YDhnn7jD', 'TimeStamp': datetime.datetime(2021, 11, 29, 9, 34, 54)}, {'Tweets': 'RT @fatoulo11: I enjoy wat

In [71]:
df = pd.DataFrame.from_dict(tweets)
df.head()

,Tweets,TimeStamp
0,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 10:05:56
1,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 09:51:47
2,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 09:49:44
3,@Mondo73410452 Good and thanks for your congra...,2021-11-29 09:35:39
4,#VisitRwanda https://t.co/96YDhnn7jD,2021-11-29 09:34:54


In [72]:
kivu_refs = ['Kivu', 'Lake Kivu', 'kivu','LakeKivu', 'lakekivu']
gorilla_refs = ['Gorillas', 'gorilla', 'Volcanoes National Park', 'volcanoes national park']
akagera_refs = ['Akagera National Park', 'akagera national park', 'akageranationalpark']
kigali_refs = ['Kigali City', 'KigaliCity', 'kigali city', 'kigalicity']
psg_refs = ['PSG','psg']
arsenal_refs = ['Arsenal','arsenal']

In [73]:
def identity(tweet, refs):
  flag = 0
  for ref in refs:
    if tweet.find(ref) != -1:
      flag =1
  return flag

df['Kivu'] = df['Tweets'].apply(lambda x: identity(x, kivu_refs))
df['Gorillas'] = df['Tweets'].apply(lambda x: identity(x, gorilla_refs))
df['Akagera'] = df['Tweets'].apply(lambda x: identity(x, akagera_refs))
df['PSG'] = df['Tweets'].apply(lambda x: identity(x, psg_refs))
df['Arsenal'] = df['Tweets'].apply(lambda x: identity(x, arsenal_refs))

df.head()

,Tweets,TimeStamp,Kivu,Gorillas,Akagera,PSG,Arsenal
0,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 10:05:56,0,0,0,0,0
1,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 09:51:47,0,0,0,0,0
2,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 09:49:44,0,0,0,0,0
3,@Mondo73410452 Good and thanks for your congra...,2021-11-29 09:35:39,0,0,0,0,0
4,#VisitRwanda https://t.co/96YDhnn7jD,2021-11-29 09:34:54,0,0,0,0,0


## 3.Preprocess

In [74]:
#import stopwords
import nltk
from nltk.corpus import stopwords

#import textblob
from textblob import Word, TextBlob

In [75]:
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('english')
custom_stopwords = ['RT', '#VisitRwanda']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [76]:
def preprocess_tweets(tweet, custom_stopwords):
  preprocessed_tweet = tweet
  preprocessed_tweet.replace('[^\w\s]', '')
  preprocessed_tweet = ' '.join(word for word in preprocessed_tweet.split() if word not in stop_words)
  preprocessed_tweet = ' '.join(word for word in preprocessed_tweet.split() if word not in custom_stopwords)
  preprocessed_tweet = ' '.join(Word(word).lemmatize() for word in preprocessed_tweet.split())

  return (preprocessed_tweet)

df['Preprocessed_Tweets'] = df['Tweets'].apply(lambda x: preprocess_tweets(x, custom_stopwords))
df.head()

,Tweets,TimeStamp,Kivu,Gorillas,Akagera,PSG,Arsenal,Preprocessed_Tweets
0,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 10:05:56,0,0,0,0,0,@fatoulo11: I enjoy watching guest fall love #...
1,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 09:51:47,0,0,0,0,0,@fatoulo11: I enjoy watching guest fall love #...
2,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 09:49:44,0,0,0,0,0,@fatoulo11: I enjoy watching guest fall love #...
3,@Mondo73410452 Good and thanks for your congra...,2021-11-29 09:35:39,0,0,0,0,0,"@Mondo73410452 Good thanks congrats, On point ..."
4,#VisitRwanda https://t.co/96YDhnn7jD,2021-11-29 09:34:54,0,0,0,0,0,https://t.co/96YDhnn7jD


## 4.Calculate Sentiment

In [84]:
df['Polarity'] = df['Preprocessed_Tweets'].apply(lambda x: TextBlob(x).sentiment[0])
df['Subjectivity'] = df['Preprocessed_Tweets'].apply(lambda x: TextBlob(x).sentiment[1])
df.head()

,Tweets,TimeStamp,Kivu,Gorillas,Akagera,PSG,Arsenal,Preprocessed_Tweets,Polarity,Subjectivity
0,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 10:05:56,0,0,0,0,0,@fatoulo11: I enjoy watching guest fall love #...,0.633333,0.466667
1,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 09:51:47,0,0,0,0,0,@fatoulo11: I enjoy watching guest fall love #...,0.633333,0.466667
2,RT @fatoulo11: I enjoy watching my guests fall...,2021-11-29 09:49:44,0,0,0,0,0,@fatoulo11: I enjoy watching guest fall love #...,0.633333,0.466667
3,@Mondo73410452 Good and thanks for your congra...,2021-11-29 09:35:39,0,0,0,0,0,"@Mondo73410452 Good thanks congrats, On point ...",0.450000,0.400000
4,#VisitRwanda https://t.co/96YDhnn7jD,2021-11-29 09:34:54,0,0,0,0,0,https://t.co/96YDhnn7jD,0.000000,0.000000


## 5.Uploading Data to BigQuery

In [78]:
#Loading necessary Libraries
from google.cloud import bigquery
import pyarrow

In [79]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Tweets               1826 non-null   object        
 1   TimeStamp            1826 non-null   datetime64[ns]
 2   Kivu                 1826 non-null   int64         
 3   Gorillas             1826 non-null   int64         
 4   Akagera              1826 non-null   int64         
 5   PSG                  1826 non-null   int64         
 6   Arsenal              1826 non-null   int64         
 7   Preprocessed_Tweets  1826 non-null   object        
 8   Polarity             1826 non-null   float64       
 9   Subjectivity         1826 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(2)
memory usage: 142.8+ KB


In [80]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [81]:
from google.cloud import bigquery

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'visit-rwanda-sentiment'
client = bigquery.Client(project=project_id)

for dataset in client.list_datasets():
  print(dataset.dataset_id)

visit_rwanda_sentiment_data


In [82]:
tableRef = client.dataset("visit_rwanda_sentiment_data").table("sentiment_dat")

job_config = bigquery.LoadJobConfig(
    schema= [
        # Specify the type of columns whose type cannot be auto-detected. For
        # example the "title" column uses pandas dtype "object", so its
        # data type is ambiguous.
        bigquery.SchemaField('Tweets', bigquery.enums.SqlTypeNames.STRING),
        bigquery.SchemaField('Preprocessed_Tweets', bigquery.enums.SqlTypeNames.STRING)
    ],

    # Optionally, set the write disposition. BigQuery appends loaded rows
    # to an existing table by default, but with WRITE_TRUNCATE write
    # disposition it replaces the table with the loaded data.
    write_disposition="WRITE_TRUNCATE",
)

bigqueryJob = client.load_table_from_dataframe(df, tableRef, job_config = job_config)
bigqueryJob.result()